In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE60690"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE60690"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE60690.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE60690.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE60690.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene Expression in Transformed Lymphocytes Reveals Phenotype-Modifying Pathways in Cystic Fibrosis"
!Series_summary	"Heritable genetic variants modify cystic fibrosis (CF) clinical phenotypes, e.g., lung disease, age-of-onset of persistent Pseudomonas aeruginosa (P. aeruginosa), and meconium ileus (MI).  Previous genome wide association studies (GWAS) have begun to inform the genetic architecture of CF phenotypes.  Analyses of gene expression will complement GWAS, as demonstrated by analyses of gene expression in lymphoblastoid cell lines (LCLs) to identify disease-related pathophysiological processes for non-CF complex traits.  In this study, global gene expression was measured in RNA from LCLs from 754 CF patients and analyzed for association with lung disease severity, age-of-onset of persistent P. aeruginosa pulmonary infection, and MI at birth.  Each phenotype displayed distinct expression associations.  Most pathways significantly associated

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the title and summary, this dataset analyzes gene expression in LCLs from CF patients
# This is not miRNA or methylation data, but gene expression analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Identifying rows
trait_row = 1  # "consortium lung phenotype" has numeric values and is related to CF lung disease severity
age_row = 2    # "age of enrollment" contains patient ages
gender_row = 0  # "Sex" has Male/Female values

# 2.2 Creating conversion functions
def convert_trait(value):
    """Convert trait (lung phenotype) value to continuous numeric value."""
    if not value or pd.isna(value):
        return None
    
    # Extract numeric value after colon
    if ":" in value:
        try:
            return float(value.split(": ")[1].strip())
        except (ValueError, IndexError):
            return None
    return None

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if not value or pd.isna(value):
        return None
    
    # Extract numeric value after colon
    if ":" in value:
        try:
            return float(value.split(": ")[1].strip())
        except (ValueError, IndexError):
            return None
    return None

def convert_gender(value):
    """Convert gender value to binary (0=female, 1=male)."""
    if not value or pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(": ")[1].strip().lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
        else:
            return None
    return None

# 3. Save Metadata
# Conduct initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The dictionary is structured with row indices as keys and lists of values as values
    sample_characteristics = {0: ['Sex: Male', 'Sex: Female'], 
                             1: ['consortium lung phenotype: 0.92156', 'consortium lung phenotype: -0.79274', 'consortium lung phenotype: 2.33374', 'consortium lung phenotype: -1.36666', 'consortium lung phenotype: 2.86073', 'consortium lung phenotype: 1.08383', 'consortium lung phenotype: 1.15792', 'consortium lung phenotype: 1.51600', 'consortium lung phenotype: -0.77528', 'consortium lung phenotype: -0.35251', 'consortium lung phenotype: 0.95176', 'consortium lung phenotype: -0.78019', 'consortium lung phenotype: -0.45468', 'consortium lung phenotype: 1.14497', 'consortium lung phenotype: -0.47971', 'consortium lung phenotype: 0.08552', 'consortium lung phenotype: -0.74197', 'consortium lung phenotype: -1.29147', 'consortium lung phenotype: 0.81747', 'consortium lung phenotype: 1.52976', 'consortium lung phenotype: 1.78443', 'consortium lung phenotype: 0.85150', 'consortium lung phenotype: 1.98295', 'consortium lung phenotype: 0.86374', 'consortium lung phenotype: 0.94161', 'consortium lung phenotype: 1.10084', 'consortium lung phenotype: -0.55910', 'consortium lung phenotype: 0.84926', 'consortium lung phenotype: 0.86596', 'consortium lung phenotype: 1.37640'],
                             2: ['age of enrollment: 38.2', 'age of enrollment: 9.2', 'age of enrollment: 22.5', 'age of enrollment: 14.4', 'age of enrollment: 33.8', 'age of enrollment: 18.6', 'age of enrollment: 27.7', 'age of enrollment: 33.5', 'age of enrollment: 17.8', 'age of enrollment: 24.1', 'age of enrollment: 16.4', 'age of enrollment: 8.7', 'age of enrollment: 10.4', 'age of enrollment: 46.3', 'age of enrollment: 19.0', 'age of enrollment: 17.1', 'age of enrollment: 10.2', 'age of enrollment: 16.7', 'age of enrollment: 39.7', 'age of enrollment: 39.3', 'age of enrollment: 25.4', 'age of enrollment: 41.2', 'age of enrollment: 18.1', 'age of enrollment: 21.5', 'age of enrollment: 17.3', 'age of enrollment: 32.4', 'age of enrollment: 19.8', 'age of enrollment: 34.3', 'age of enrollment: 25.3', 'age of enrollment: 46.7']}
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame()
    
    # For each sample characteristic row
    for row_idx, values in sample_characteristics.items():
        # Create a temporary Series for this row
        row_data = pd.Series(values, name=row_idx)
        
        # Add to the DataFrame
        clinical_data = pd.concat([clinical_data, row_data.to_frame().T])
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview_result = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    for key, value in preview_result.items():
        print(f"{key}: {value}")
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
0: [0.92156, 38.2, 1.0]
1: [-0.79274, 9.2, 0.0]
2: [2.33374, 22.5, nan]
3: [-1.36666, 14.4, nan]
4: [2.86073, 33.8, nan]
5: [1.08383, 18.6, nan]
6: [1.15792, 27.7, nan]
7: [1.516, 33.5, nan]
8: [-0.77528, 17.8, nan]
9: [-0.35251, 24.1, nan]
10: [0.95176, 16.4, nan]
11: [-0.78019, 8.7, nan]
12: [-0.45468, 10.4, nan]
13: [1.14497, 46.3, nan]
14: [-0.47971, 19.0, nan]
15: [0.08552, 17.1, nan]
16: [-0.74197, 10.2, nan]
17: [-1.29147, 16.7, nan]
18: [0.81747, 39.7, nan]
19: [1.52976, 39.3, nan]
20: [1.78443, 25.4, nan]
21: [0.8515, 41.2, nan]
22: [1.98295, 18.1, nan]
23: [0.86374, 21.5, nan]
24: [0.94161, 17.3, nan]
25: [1.10084, 32.4, nan]
26: [-0.5591, 19.8, nan]
27: [0.84926, 34.3, nan]
28: [0.86596, 25.3, nan]
29: [1.3764, 46.7, nan]
Clinical data saved to: ../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE60690.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 97
Header line: "ID_REF"	"GSM1485230"	"GSM1485231"	"GSM1485232"	"GSM1485233"	"GSM1485234"	"GSM1485235"	"GSM1485236"	"GSM1485237"	"GSM1485238"	"GSM1485239"	"GSM1485240"	"GSM1485241"	"GSM1485242"	"GSM1485243"	"GSM1485244"	"GSM1485245"	"GSM1485246"	"GSM1485247"	"GSM1485248"	"GSM1485249"	"GSM1485250"	"GSM1485251"	"GSM1485252"	"GSM1485253"	"GSM1485254"	"GSM1485255"	"GSM1485256"	"GSM1485257"	"GSM1485258"	"GSM1485259"	"GSM1485260"	"GSM1485261"	"GSM1485262"	"GSM1485263"	"GSM1485264"	"GSM1485265"	"GSM1485266"	"GSM1485267"	"GSM1485268"	"GSM1485269"	"GSM1485270"	"GSM1485271"	"GSM1485272"	"GSM1485273"	"GSM1485274"	"GSM1485275"	"GSM1485276"	"GSM1485277"	"GSM1485278"	"GSM1485279"	"GSM1485280"	"GSM1485281"	"GSM1485282"	"GSM1485283"	"GSM1485284"	"GSM1485285"	"GSM1485286"	"GSM1485287"	"GSM1485288"	"GSM1485289"	"GSM1485290"	"GSM1485291"	"GSM1485292"	"GSM1485293"	"GSM1485294"	"GSM1485295"	"GSM1485296"	"GSM1485297"	"GSM1485298"	"GSM1485299"	"GSM1485300"	"GSM1485301"	"GSM1485302"	

Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers in the dataset
# The IDs appear to be numeric probe identifiers (e.g., "2315554")
# These are not standard human gene symbols and would need mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determining the mapping between identifiers in the gene expression data and gene annotations
# From the previous steps, we saw:
# - Gene expression data has numeric identifiers in the 'ID' column (e.g., "2315554")
# - Gene annotation data has matching IDs and gene information in the 'gene_assignment' column

# First, let's check the structure of both datasets
print("Gene data index examples (first 5):", list(gene_data.index[:5]))
print("Gene annotation ID examples (first 5):", list(gene_annotation['ID'][:5]))

# 2. Create mapping dataframe using the built-in function for extracting human gene symbols
mapping_df = pd.DataFrame()
mapping_df['ID'] = gene_annotation['ID']
mapping_df['Gene'] = gene_annotation['gene_assignment'].apply(extract_human_gene_symbols)

# Check for ID intersection between gene data and mapping
gene_ids_in_data = set(gene_data.index)
gene_ids_in_mapping = set(mapping_df['ID'])
common_ids = gene_ids_in_data.intersection(gene_ids_in_mapping)
print(f"Number of common IDs between gene data and mapping: {len(common_ids)}")

# Print statistics to verify mapping looks reasonable
print(f"Total number of probes in mapping: {len(mapping_df)}")
print(f"Number of probes with gene mappings: {mapping_df['Gene'].apply(len).gt(0).sum()}")

# Filter the mapping to only include IDs present in gene_data
filtered_mapping = mapping_df[mapping_df['ID'].isin(gene_data.index)]
print(f"Number of probes in filtered mapping: {len(filtered_mapping)}")
print(f"Number of probes with gene mappings after filtering: {filtered_mapping['Gene'].apply(len).gt(0).sum()}")

# 3. Apply gene mapping to convert probe-level measurements to gene expression using filtered mapping
gene_data = apply_gene_mapping(gene_data, filtered_mapping)

# Print the shape of the resulting gene expression data
print(f"Shape of gene expression data: {gene_data.shape}")
if not gene_data.empty:
    print("First 5 gene symbols:")
    print(gene_data.index[:5])
else:
    print("Warning: Gene expression data is empty after mapping")

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Gene data index examples (first 5): ['2315554', '2315633', '2315674', '2315739', '2315894']
Gene annotation ID examples (first 5): ['2315100', '2315106', '2315109', '2315111', '2315113']


Number of common IDs between gene data and mapping: 12033
Total number of probes in mapping: 9390555


Number of probes with gene mappings: 31980
Number of probes in filtered mapping: 9084915


Number of probes with gene mappings after filtering: 11962


Shape of gene expression data: (0, 754)
Gene expression data saved to: ../../output/preprocess/Cystic_Fibrosis/gene_data/GSE60690.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Use the clinical data that was already processed and saved in previous steps
selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased
# First check if trait is biased
trait_type = 'binary' if len(linked_data[trait].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_trait_biased = judge_binary_variable_biased(linked_data, trait)
else:
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait)

# Check for and remove biased demographic features
unbiased_linked_data = linked_data.copy()
if "Age" in unbiased_linked_data.columns:
    age_biased = judge_continuous_variable_biased(unbiased_linked_data, 'Age')
    if age_biased:
        print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Age')
    else:
        print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")

if "Gender" in unbiased_linked_data.columns:
    gender_biased = judge_binary_variable_biased(unbiased_linked_data, 'Gender')
    if gender_biased:
        print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
        unbiased_linked_data = unbiased_linked_data.drop(columns='Gender')
    else:
        print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

# 5. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data comparing CFBE41o-ΔF508 (CF) cells with CFBE41o−CFTR (control) cells."
)

# 6. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (0, 754)
First few genes with their expression values after normalization:
Empty DataFrame
Columns: [GSM1485230, GSM1485231, GSM1485232, GSM1485233, GSM1485234, GSM1485235, GSM1485236, GSM1485237, GSM1485238, GSM1485239, GSM1485240, GSM1485241, GSM1485242, GSM1485243, GSM1485244, GSM1485245, GSM1485246, GSM1485247, GSM1485248, GSM1485249, GSM1485250, GSM1485251, GSM1485252, GSM1485253, GSM1485254, GSM1485255, GSM1485256, GSM1485257, GSM1485258, GSM1485259, GSM1485260, GSM1485261, GSM1485262, GSM1485263, GSM1485264, GSM1485265, GSM1485266, GSM1485267, GSM1485268, GSM1485269, GSM1485270, GSM1485271, GSM1485272, GSM1485273, GSM1485274, GSM1485275, GSM1485276, GSM1485277, GSM1485278, GSM1485279, GSM1485280, GSM1485281, GSM1485282, GSM1485283, GSM1485284, GSM1485285, GSM1485286, GSM1485287, GSM1485288, GSM1485289, GSM1485290, GSM1485291, GSM1485292, GSM1485293, GSM1485294, GSM1485295, GSM1485296, GSM1485297, GSM1485298, GSM1485299, GSM1485300, GSM1485301, GSM1485